In [1]:
import gensim
isReload = int(input("Type 1 to initiate reload of google word2vec: "))

if isReload:
    model_word2vec_temp = gensim.models.KeyedVectors.load_word2vec_format('/home/owner/デスクトップ/milnet+edu/GoogleNews-vectors-negative300.bin', binary=True)  
    model_word2vec = model_word2vec_temp

print('word2vec model loaded')


Type 1 to initiate reload of google word2vec: 1
word2vec model loaded


In [2]:
import sys, os
sys.path.append(os.pardir)

os.environ["KERAS_BACKEND"]='tensorflow'
#import glob
import numpy as np


import keras
from keras.layers import Input, merge
from keras.models import Model
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import Embedding
from keras.layers import LSTM
import keras.backend as K
from keras.layers import Lambda, regularizers, Average

from keras.models import Model
from keras.layers import Input, Conv2D, Conv1D, MaxPooling2D, GlobalMaxPooling2D, GlobalMaxPooling1D, MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import GRU
from keras.layers.wrappers import Bidirectional, TimeDistributed
from keras.layers.core import Dropout, Dense, Lambda, Masking
from keras.layers import merge, Layer, Activation, Dot, Concatenate, Flatten, Lambda

from keras.initializers import Identity,glorot_normal
from keras import regularizers

from keras import metrics

from keras.utils import plot_model



/home/owner/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/owner/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [33]:
x_test = np.load('/home/owner/デスクトップ/milnet+edu/data/Preprocessed/test_data_features.npy')#np.load('/home/owner/デスクトップ/PythonFile/imdb/t_train.npy')
y_test = np.load('/home/owner/デスクトップ/milnet+edu/data/Preprocessed/test_data_labels.npy')#np.load('/home/owner/デスクトップ/PythonFile/imdb/t_test.npy')
print(x_test)
print(x_test.shape)


[list([[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 226847, 572747, 548, 321, 3051], [0, 0, 0, 0, 0, 166, 37, 107, 2, 359991, 53, 515, 14, 70882, 8352], [0, 0, 0, 0, 0, 0, 0, 37, 5385, 31769, 49, 1, 138, 4096, 10904], [0, 0, 0, 0, 0, 0, 0, 0, 11, 585, 21395, 35, 138, 286, 2351], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 37, 1881, 3006], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 14, 610, 11, 2485], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 459, 1141, 1, 30, 113], [0, 0, 0, 0, 0, 0, 0, 0, 37, 76, 7276, 18, 205, 1280, 1453], [0, 0, 0, 0, 0, 0, 0, 0, 131, 82, 4007, 465, 27026, 124, 2579], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3041, 29585, 3713], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 50754, 2, 11, 2652], [0, 0, 0, 0, 0, 131, 488, 261, 11, 3304, 21068, 1, 597, 11, 2652], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 58, 891, 150, 773], [0, 0, 0, 0, 0, 0, 0, 4373, 11, 3117, 70882, 742, 82, 745, 6640], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 126, 10117, 4], [0, 0, 0, 0, 3, 52, 2579, 12, 28, 59, 336, 21, 42, 891, 706], [0, 0, 0, 0, 0, 0, 14, 11, 70882

In [4]:
x_train = np.load('/home/owner/デスクトップ/milnet+edu/data/Preprocessed/train_data_features.npy')#np.load('/home/owner/デスクトップ/PythonFile/imdb/x_train_sort.npy')
y_train = np.load('/home/owner/デスクトップ/milnet+edu/data/Preprocessed/train_data_labels.npy')#np.load('/home/owner/デスクトップ/PythonFile/imdb/x_test_sort.npy')
print(len(x_train))

x_test = np.load('/home/owner/デスクトップ/milnet+edu/data/Preprocessed/test_data_features.npy')#np.load('/home/owner/デスクトップ/PythonFile/imdb/t_train.npy')
y_test = np.load('/home/owner/デスクトップ/milnet+edu/data/Preprocessed/test_data_labels.npy')#np.load('/home/owner/デスクトップ/PythonFile/imdb/t_test.npy')

x_valid = np.load('/home/owner/デスクトップ/milnet+edu/data/Preprocessed/validation_data_features.npy')
y_valid = np.load('/home/owner/デスクトップ/milnet+edu/data/Preprocessed/validation_data_labels.npy')

train_idx = np.load('/home/owner/デスクトップ/milnet+edu/data/Preprocessed/train_data_idx_binary_only.npy')
test_idx = np.load('/home/owner/デスクトップ/milnet+edu/data/Preprocessed/test_data_idx_binary_only.npy')
valid_idx = np.load('/home/owner/デスクトップ/milnet+edu/data/Preprocessed/validation_data_idx_binary_only.npy')

#print(len(train_idx))
x_train = x_train[train_idx]
y_train = y_train[train_idx]
print(len(x_train))
print(len(y_train))

x_test = x_test[test_idx]
y_test = y_test[test_idx]

x_valid = x_valid[valid_idx]
y_valid = y_valid[valid_idx]
#print(len(x_train))

word_idx=np.load( '/home/owner/デスクトップ/milnet+edu/data/Preprocessed/' + 'vocab_idx.npy')
#print(idx)
embWeights=np.load( '/home/owner/デスクトップ/milnet+edu/data/Preprocessed/' + 'weights.npy')#np.load('/home/owner/デスクトップ/PythonFile/imdb/weights.npy')

print(len(word_idx))
print(y_test)

print('data loaded')

281926
210642
210642
53831
[0 0 0 ... 9 9 9]
data loaded


In [5]:
numSentencesPerDoc, numWordsPerSentence = x_train[0].shape[0], x_train[0].shape[1]
print(numSentencesPerDoc, numWordsPerSentence)
#print(x_train[0])

vocabSize, embeddingSize = embWeights.shape[0], embWeights.shape[1]
print(vocabSize, embeddingSize)

dropWordEmb = 0.25
recursiveClass = GRU

filters = 1 #embeddingSize*2
windowMin = 3
windowMax = 7
# dimOfSentimentMetrics = 5
batch_size = 256
epochs = 25
numGRU = 100
numDensePool=50
eta = 1e-5
dr = 0.5

##

#wordsInputs = Input(shape=(numWordsPerSentence,1), batch_shape=(numSentencesPerDoc,numWordsPerSentence,), dtype='int32', name='words_input')

x_in = Input( shape = ( numSentencesPerDoc, numWordsPerSentence ) , name='Input' )
#x_pop = Lambda( lambda x: x, output_shape=(numWordsPerSentence, ) , name='convert_shape' )( x_in )
    
#Layer functionの定義
embLayer = Embedding( input_dim=embWeights.shape[0], output_dim=embWeights.shape[1], weights=[embWeights]
                      ,mask_zero=True , trainable=True, embeddings_regularizer=regularizers.l2(0.0000001)
                      , input_length=numWordsPerSentence, name='Embedding' )


#embLayer = model_word2vec.get_keras_embedding()

maxPooledPerDoc = []
convNets = []
maxPools = []

extraDimLayer = Lambda(lambda x: K.expand_dims(x), name='extraDimForConvo')
squeezeThirdLayer = Lambda(lambda x: K.squeeze(x, 3), name='squeezeThirdLayer')

for windowSize in range(windowMin,windowMax):
    name='word_mat_convo_win_size_'+str(windowSize)
    convNet = Conv2D(filters, kernel_size=(windowSize,embeddingSize), padding='valid', activation='relu'
                     ,strides=1, use_bias=True, input_shape=(numWordsPerSentence, embeddingSize, 1), data_format="channels_last",kernel_initializer=glorot_normal()
                     ,bias_regularizer=regularizers.l2(eta), kernel_regularizer=regularizers.l2(eta),name=name)
    convNets.append(convNet)
    name='word_mat_max_pool_win_size_'+str(windowSize)
    maxPool = MaxPooling1D(pool_size = int(numWordsPerSentence-windowSize-1), padding='valid')
    maxPools.append(maxPool)
    
    
for i in range(numSentencesPerDoc):
    maxPooledPerSentence = []
    x_pop = Lambda(lambda x: x[:,i], output_shape=(numWordsPerSentence, ) , name='convert_shape_'+'sentence'+str(i+1))( x_in )

    for j in range(windowMax-windowMin):   
        emb = embLayer(x_pop)
        emb = Dropout(dr,name='DropEmb'+str(i)+str(j))(emb)
        reshaped = extraDimLayer(emb)#Lambda(lambda x: K.expand_dims(x), name='extraDimForConvo_'+str(j)+'_sentence_'+str(i))(emb)
        name='word_mat_convo_win_size_'+str(j)+'_sentence_'+str(i)
        # wordsCNN = Conv2D(filters, kernel_size=(windowSize,embeddingSize), padding='valid', 
        #                    activation='relu', strides=1, use_bias=True, input_shape=(numWordsPerSentence, embeddingSize, 1), data_format="channels_last",
        #                    kernel_initializer=glorot_normal(),kernel_regularizer=regularizers.l2(),name=name)(reshaped)
        wordsCNN  = convNets[j](reshaped)
        wordsCNN = Dropout(dr,name='DropCNN'+str(i)+str(j))(wordsCNN)
        squeezed = squeezeThirdLayer(wordsCNN)#Lambda(lambda x: K.squeeze(x, 3), name='squeezeThirdLayer_'+str(j)+'_sentence_'+str(i))(wordsCNN)
        # newShape = (-1, int(squeezed.shape[1])*int(squeezed.shape[2]))
        # squeezed = Lambda(lambda x: K.reshape(x,shape=newShape), name ='squeezeDimForMaxPool'+str(i)+str(j))(squeezed)
        wordsCNNPooled=GlobalMaxPooling1D()(squeezed)
        #wordsCNNPooled= MaxPooling1D(pool_size = int(squeezed.shape[1]), padding='valid')(squeezed)
        maxPooledPerSentence.append(wordsCNNPooled)
        
    mergedPoolForSentence = Concatenate(axis = 1)(maxPooledPerSentence)
    newShape=(-1,1,int(mergedPoolForSentence.shape[1]))
    reshapedPoolForSentence = Lambda(lambda x: K.reshape(x,shape=newShape), name ='switch_axis_'+'sentence'+str(i+1)+'winSize'+str(j+windowMin))(mergedPoolForSentence)
    densePoolForSentence = Dense(numDensePool, bias_regularizer=regularizers.l2(eta),
                                 kernel_regularizer=regularizers.l2(eta), activation='softmax', use_bias=True)(reshapedPoolForSentence)

    densePoolForSentence = Dropout(dr,name='DropDense'+str(i))(densePoolForSentence)
    maxPooledPerDoc.append(densePoolForSentence)
    
#Naive Approach
averaged = Average()(maxPooledPerDoc) 
averaged = Lambda(lambda x:K.reshape(x,shape=(-1,int(averaged.shape[1])*int(averaged.shape[2]))), name ='attend_output')(averaged)
out_avg = Dense(1, activation='sigmoid', use_bias=True)(averaged) 
    
#Apply Attention 
mergedPoolPerDoc = Concatenate(axis = 1)(maxPooledPerDoc)
biRnn_ = Bidirectional(GRU(int(mergedPoolPerDoc.shape[2]),  return_sequences=True, bias_regularizer=regularizers.l2(eta)
                           ,kernel_regularizer=regularizers.l2(eta),recurrent_regularizer=regularizers.l2(eta)
                           ,dropout=dr, recurrent_dropout=dr, unroll=True), merge_mode='concat')(mergedPoolPerDoc)
newShape = (-1, int(mergedPoolPerDoc.shape[1]), 2*int(mergedPoolPerDoc.shape[2]))
biRnn = Lambda(lambda x: K.reshape(x,shape=newShape), name ='biRnn_TF_Reminder')(biRnn_)

CONTEXT_DIM = 100#int(int(biRnn.shape[1])*int(biRnn.shape[2])/2) 

eij = Dense(CONTEXT_DIM, kernel_regularizer=regularizers.l2(eta), bias_regularizer=regularizers.l2(eta)
            , use_bias=True, activation='tanh')(biRnn)
eij = Dense(CONTEXT_DIM, kernel_regularizer=regularizers.l2(eta), use_bias=False, activation='softmax')(eij)

weighted_input_ = Dot(axes = 1)([eij, biRnn])
weighted_input = Lambda(lambda x: K.reshape(x,shape=(-1,int(weighted_input_.shape[1])*int(weighted_input_.shape[2]))), name ='attend_output')(weighted_input_)

out = Dense(1, kernel_regularizer=regularizers.l2(eta), bias_regularizer=regularizers.l2(eta)
            , activation='sigmoid', use_bias=True)(weighted_input)




##

model = Model(input=[x_in], output=[out])
# adadelta = keras.optimizers.Adadelta(lr=0.5, rho=0.95, epsilon=None, decay=0.0)
# model.compile(loss='binary_crossentropy',
#               optimizer=adadelta,
#               metrics=['accuracy'])
         
model.compile(loss='binary_crossentropy',
              optimizer=keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0),
              metrics=['accuracy'])

print("Attention Model Build Complete")
##
model_avg = Model(inputs=[x_in], outputs=[out_avg])
model_avg.compile(loss='binary_crossentropy',
              optimizer=keras.optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0),
              metrics=['accuracy'])

print("Average Model Build Complete")
##
#save model to png file
from keras.utils import plot_model
plot_model( model, to_file='model.png' )

#モデルを保存せず直接可視化
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
SVG( model_to_dot( model ).create( prog='dot', format='svg' ) )
'''
##
print('Train...')
history = model.fit(x_train, y_train, batch_size = batch_size, verbose=1, epochs=epochs
                    ,validation_split=0.2, shuffle=True)
'''
                    
                    
                    

6 15
53831 300


ResourceExhaustedError: OOM when allocating tensor with shape[53831,300]
	 [[Node: Embedding/embeddings/Assign = Assign[T=DT_FLOAT, _class=["loc:@Embedding/embeddings"], use_locking=true, validate_shape=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](Embedding/embeddings, Embedding/random_uniform)]]

Caused by op 'Embedding/embeddings/Assign', defined at:
  File "/home/owner/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/owner/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/owner/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/owner/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/owner/anaconda3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()


In [ ]:
x_train.shape

In [ ]:
save_model = keras.callbacks.ModelCheckpoint('./params.hdf5', monitor='val_loss', verbose=1
                                             , save_best_only=True, save_weights_only=True, mode='min', period=1)

In [ ]:
history = model.fit(x_train, y_train, batch_size = batch_size, verbose=1, epochs=200 #epochs
                        ,validation_data=(x_valid,y_valid), shuffle=True, callbacks=[save_model])

In [ ]:
score=model.evaluate(x_test, y_test)
print(score[0],score[1])

In [ ]:
history2 = model.fit(x_train, y_train, batch_size = batch_size, verbose=1, initial_epochs = 200, epochs=300 #epochs
                        ,validation_data=(x_valid,y_valid), shuffle=True, callbacks=[save_model])

In [ ]:
score=model.evaluate(x_test, y_test)
print(score[0],score[1])

In [ ]:
history3 = model.fit(x_train, y_train, batch_size = batch_size, verbose=1, initial_epochs = 300, epochs=500 #epochs
                        , validation_data=(x_valid,y_valid), shuffle=True, callbacks=[save_model])

In [ ]:
#plot history
import matplotlib.pyplot as plt
def plot_history(history):
    # 精度の履歴をプロット
    plt.plot(history.history['acc'],label="accuracy")
    plt.plot(history.history['val_acc'],"o-",label="val_acc")
    plt.title('model accuracy')
    plt.xlabel('epoch')
    plt.ylabel('accuracy')
    plt.legend(loc="lower right")
    plt.show()

    # 損失の履歴をプロット
    plt.plot(history.history['loss'],label="loss",)
    plt.plot(history.history['val_loss'],"o-",label="val_loss")
    plt.title('model loss')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.legend(loc='lower right')
    plt.show()
    

In [ ]:
plot_history(history)

In [ ]:
#model_avg.save_weights('./params.hdf5')

In [ ]:
score=model.evaluate(x_test, y_test)

In [ ]:
print(score[0],score[1])

In [ ]:
print('h')